In [1]:
# по номерам
def mul(x,y):
    assert len(x)==len(y)
    return tuple((y[i] for i in x))
def inv(x):
    y = [-1]*len(x)
    for i in range(len(x)):
        y[x[i]]=i
    return tuple(y)

In [2]:
def from_cycle(cycles,size):
    x = [i for i in range(size)]
    if len(cycles)>0 and type(cycles[0])==int:
        cycles = [cycles]
    #print(cycles)
    for cycle in cycles:
        for i in range(len(cycle)-1):
            s = cycle[i]
            d = cycle[i+1]
            x[s] = d
        x[cycle[-1]] = cycle[0]
    return tuple(x)

In [3]:
def inv(x):
    y = [-1]*len(x)
    for i in range(len(x)):
        y[x[i]]=i
    return tuple(y)

In [4]:
# по позициям
def mul(x,y):
    assert len(x)==len(y)
    z = [-1]*len(x)
    for i in range(len(y)):
        z[i]=x[y[i]]
    return tuple(z)

In [5]:
def sims_from_rel(rel):
    for i in range(len(rel)):
        yield (rel,'')
        for j in range(1,(len(rel)+1)//2):
            yield (rel[:-j],rel[-1:-j-1:-1].swapcase())
        rel = rel[1:]+rel[0]
    rel = rel[::-1].swapcase()
    for i in range(len(rel)):
        yield (rel,'')
        for j in range(1,(len(rel)+1)//2):
            yield (rel[:-j],rel[-1:-j-1:-1].swapcase())
        rel = rel[1:]+rel[0]


In [6]:
rel = 'ababab'
j=2
rel[-1:-j-1:-1]

'ba'

In [7]:
{x for x in sims_from_rel('ababab')}

{('ABAB', 'ba'),
 ('ABABA', 'b'),
 ('ABABAB', ''),
 ('BABA', 'ab'),
 ('BABAB', 'a'),
 ('BABABA', ''),
 ('abab', 'BA'),
 ('ababa', 'B'),
 ('ababab', ''),
 ('baba', 'AB'),
 ('babab', 'A'),
 ('bababa', '')}

In [8]:
{x for x in sims_from_rel('aaa')}

{('AA', 'a'), ('AAA', ''), ('aa', 'A'), ('aaa', '')}

In [9]:
def symsims_from_rel(rel):
    if len(rel)%2 !=0: return 
    for i in range(len(rel)):
        j = len(rel)//2 
        a = rel[:-j]
        b = rel[-1:-j-1:-1].swapcase()
        if a>b:  yield (a,b)
        elif a<b:  yield (b,a)
        rel = rel[1:]+rel[0]


In [10]:
{k:v for k,v in symsims_from_rel('abcd')}

{'ab': 'DC', 'bc': 'AD', 'cd': 'BA', 'da': 'CB'}

In [11]:
{k:v for k,v in symsims_from_rel('aaaa')}

{'aa': 'AA'}

In [12]:
def standardize_rel(rel):
    minrel = rel
    for i in range(len(rel)):
        if rel<minrel: minrel = rel
        rel = rel[1:]+rel[0]
    rel = rel[::-1].swapcase()
    for i in range(len(rel)):
        if rel<minrel: minrel = rel
        rel = rel[1:]+rel[0]
    return minrel

In [13]:
import copy
def listind(l,p,void=None):
    while len(l)<=p: l.append(copy.copy(void))
    #print(l)
    return l[p]

In [14]:
import re
def simplify(sims,symsims,op,stop):
    simplified = True
    while simplified:
        simplified = False
        for i in range(len(op)+1):
            if i<len(sims):
                for f,t in sims[i].items():
                    op,n = re.subn(f,t,op)
                    if n>0: simplified = True
                    if len(op)<stop: break
            if i<len(symsims):
                for f,t in symsims[i].items():
                    op,n = re.subn(f,t,op)
                    if n>0: simplified = True
                    if len(op)<stop: break
            if len(op)<stop: break
        if len(op)<stop: break
    return op
    

In [15]:
sgens = 'ab'
rels = {'aaa','bbbb','abab'}
gens = set()
for g in sgens:
    rels.add(standardize_rel(g+g[::-1].swapcase()))
    gens.add(g)
    gens.add(g[::-1].swapcase())
# строим упрощающие конструкции
sims = []
symsims = []
for rel in rels:
    for k,v in sims_from_rel(rel):
        listind(sims,len(k),{})[k] = v
    for k,v in symsims_from_rel(rel):
        listind(symsims,len(k),{})[k] = v
simplify(sims,symsims,'ABaaB',0)

''

In [16]:
import re
def group_from_rels(sgens,rels,stoplen):
    # sgens - строка, rels - set
    # добавляем обратные элементы в генераторы, и соотношения об этом
    gens = set()
    for g in sgens:
        rels.add(standardize_rel(g+g[::-1].swapcase()))
        gens.add(g)
        gens.add(g[::-1].swapcase())
    # строим упрощающие конструкции
    sims = []
    symsims = []
    for rel in rels:
        for k,v in sims_from_rel(rel):
            listind(sims,len(k),{})[k] = v
        for k,v in symsims_from_rel(rel):
            listind(symsims,len(k),{})[k] = v
    print(sims,symsims)
    # поехали
    ops = [{''}]
    for it in range(1,stoplen):
        print(it)
        ops.append(set())
        for op1 in ops[it-1]:
            for g in gens:
                # рассматриваем оператор
                op=op1+g
#                print('рассматриваем',op)
                # пытаемся его упростить
                op = simplify(sims,symsims,op,it)
                if len(op)<it: 
#                    print('упрощено')
                    continue
                # упростить не получилось, сравниваем со всеми уже полученными операторами
                found = False
                for it2 in range(1,it+1):
                    for op2 in ops[it2]:
                        test = op+(op2[::-1].swapcase())
#                        print(test)
                        test = simplify(sims,symsims,test,0)
                        if test=='': # симплифицировали
                            found = True
                            print(op,'==',op2,' -> ',op+(op2[::-1].swapcase()))
                            break
                    if found: break
                if not found:
                    ops[it].add(op)
#                    print('   добавляем')
    return ops    
                

In [17]:
group_from_rels('ab',{'aa','bb','ababab'},7)

[{}, {}, {'aa': '', 'AA': '', 'Aa': '', 'aA': '', 'Bb': '', 'bB': '', 'bb': '', 'BB': ''}, {}, {'abab': 'BA', 'baba': 'AB', 'BABA': 'ab', 'ABAB': 'ba'}, {'ababa': 'B', 'babab': 'A', 'BABAB': 'a', 'ABABA': 'b'}, {'ababab': '', 'bababa': '', 'BABABA': '', 'ABABAB': ''}] [{}, {'a': 'A', 'b': 'B'}, {}, {'aba': 'BAB', 'bab': 'ABA'}]
1
B == B  ->  Bb
A == A  ->  Aa
2
BA == BA  ->  BAab
AB == AB  ->  ABba
3
BAB == BAB  ->  BABbab
ABA == BAB  ->  ABAbab
ABA == BAB  ->  ABAbab
4
5
6


[{''}, {'A', 'B'}, {'AB', 'BA'}, {'BAB'}, set(), set(), set()]

In [18]:
group_from_rels('ab',{'aaa','bbbb','abab','aBaBaBaB'},9)

[{}, {}, {'Aa': '', 'aA': '', 'Bb': '', 'bB': '', 'aa': 'A', 'AA': 'a'}, {'bbb': 'B', 'BBB': 'b', 'aba': 'B', 'bab': 'A', 'BAB': 'a', 'ABA': 'b', 'aaa': '', 'AAA': ''}, {'bbbb': '', 'BBBB': '', 'abab': '', 'baba': '', 'BABA': '', 'ABAB': ''}, {'aBaBa': 'bAb', 'BaBaB': 'AbA', 'bAbAb': 'aBa', 'AbAbA': 'BaB'}, {'aBaBaB': 'bA', 'BaBaBa': 'Ab', 'bAbAbA': 'aB', 'AbAbAb': 'Ba'}, {'aBaBaBa': 'b', 'BaBaBaB': 'A', 'bAbAbAb': 'a', 'AbAbAbA': 'B'}, {'aBaBaBaB': '', 'BaBaBaBa': '', 'bAbAbAbA': '', 'AbAbAbAb': ''}] [{}, {}, {'bb': 'BB', 'ab': 'BA', 'ba': 'AB'}, {}, {'bAbA': 'aBaB', 'BaBa': 'AbAb'}]
1
2
BA == BA  ->  BAab
BB == BB  ->  BBbb
AB == AB  ->  ABba
3
aBB == BAb  ->  aBBBab
aBA == Ab  ->  aBABa
BBa == bAB  ->  BBabaB
ABa == bA  ->  ABaaB
ABB == ABB  ->  ABBbba
AbA == aBa  ->  AbAAbA
BBA == BBA  ->  BBAabb
4
ABBA == bAb  ->  ABBABaB
ABBa == aBaB  ->  ABBabAbA
aBaB == aBaB  ->  aBaBbAbA
BaBA == BAb  ->  BaBABab
AbAb == aBBA  ->  AbAbabbA
BBAb == BaBB  ->  BBAbbbAb
bABB == bABB  ->  bABBbbaB
b

[{''},
 {'A', 'B', 'a', 'b'},
 {'AB', 'Ab', 'BA', 'BB', 'Ba', 'aB', 'bA'},
 {'ABB', 'BAb', 'BBA', 'BaB', 'aBa', 'bAB', 'bAb'},
 {'BaBB', 'aBBA', 'aBaB', 'bABB'},
 {'BaBBa', 'aBBAb', 'bABBA', 'bABBa'},
 {'BaBBaB', 'bABBAb'},
 {'BaBAbAb', 'BaBBaBA', 'BaBBaBB', 'bABBAbA'},
 {'BaBAbABB', 'BaBBaBAb', 'BaBBaBBa', 'bABBAbAB', 'bABBAbAb'}]

In [19]:
def gen_rels(gens,stoplen):
    # добавляем обратные элементы в генераторы
    invgens = {}
    for g,gn in gens.items():
        invgens[inv(g)]=gn[::-1].swapcase()
    for g,gn in invgens.items():
        if g not in gens:
            gens[g]=gn
    # и соотношения об этом
    rels = set()
    for g,n in gens.items():
        rels.add(standardize_rel(n+n[::-1].swapcase()))
    sims = []
    symsims = []
    for rel in rels:
        for k,v in sims_from_rel(rel):
            listind(sims,len(k),{})[k] = v
        for k,v in symsims_from_rel(rel):
            listind(symsims,len(k),{})[k] = v
    # поехали
    ops = [{tuple(i for i in range(len(next(iter(gens))))):''}]
#    print(ops)
    for it in range(1,stoplen):
#        print('---',it,'---')
        ops.append({})
        for op1,n1 in ops[it-1].items():
            for g,gn in gens.items():
                op = mul(op1,g)
                n = n1+gn
                n = simplify(sims,symsims,n,it)
#                print('рассматриваю',n1,gn,'=',n)
                if len(n)<it:
                    if op in ops[len(n)]: continue
                    found = False
                    for i in range(it,-1,-1):
                        if op in ops[i]:
                            found = True
                            print(n1,gn,'->',n,'==',ops[i][op])
                            break
                    assert found,(n1,gn,n,op)
                    continue # упростилось
                    
#                print(it,it-1)
                if op in ops[it] or op in ops[it-1]:
                    # нашли в текущем или предыдущем
                    if op in ops[it]:  
                        n2 = ops[it][op]
                    else:
                        n2 = ops[it-1][op]
                    
                    rel = simplify(sims,symsims, n+n2[::-1].swapcase() ,0)
                    if rel=='': continue
                    rels.add(standardize_rel(rel))
                    for k,v in sims_from_rel(rel):
                        listind(sims,len(k),{})[k] = v
                    for k,v in symsims_from_rel(rel):
                        listind(symsims,len(k),{})[k] = v
                    continue
                    
                found = False
                for i in range(it-2,-1,-1):
#                    print(i)
                    if op in ops[i]:
                        found = True
                        # нашли в прошлых
                        n2 = ops[i][op]
                        rel = simplify(sims,symsims, n+n2[::-1].swapcase() ,0)
                        print(n,'==',n2,' -> ',rel)
                        if rel=='': break
                        rels.add(standardize_rel(rel))
                        for k,v in sims_from_rel(rel):
                            listind(sims,len(k),{})[k] = v
                        for k,v in symsims_from_rel(rel):
                            listind(symsims,len(k),{})[k] = v
                        break
                if found: continue
                
                # новый элемент
#                print('добавляю',op)
                ops[it][op]=n
    
    return (rels,ops)

In [20]:
gen_rels({(1,2,0):'a'},5)

({'AAA', 'Aa'},
 [{(0, 1, 2): ''}, {(1, 2, 0): 'a', (2, 0, 1): 'A'}, {}, {}, {}])

In [26]:
rels,ops = gen_rels({
    from_cycle((0,1,2),4):'A',
    inv(from_cycle((0,1,2,3),4)):'B',
},10)

AbA A -> Aba == aB
ABBa a -> ABBA == bAb
AbAb A -> BAAB == BaB
AbAb a -> AbaB == BAb
BBaBB == bAb  ->  
ABBaBB == AbAb  ->  
ABBaBa == BBAb  ->  


In [28]:
[len(lops) for lops in ops]

[1, 4, 7, 7, 4, 1, 0, 0, 0, 0]

In [23]:
rels

{'AAA', 'ABAB', 'ABaBaBAb', 'Aa', 'BBBB', 'Bb'}

TODO
* проверять различные траектории упрощений
* * ? что могут дать усложнения?
* давать номера элементам
* по соотношениям:
* * таблица умножения
* по группе:
* * сделать символьное умножение
* * какие будут соотношения при других генераторах?
* * генерация графа Кэли

рассмотреть группы и их подгруппы и их произведения (прямое, полупрямое, ...):
* просто перестановок: 3,4,5,6,7,8,9,10,11,12,13
* тетраэр, куб/октаэдр, додекаэдр/икосаэдр
* группы 4-мерных платоновых тел
* пирамидка Рубика, кубик Рубика 2х2, кубик рубика 3х3